# 导入相关库、数据读取、集合与参数定义、决策变量定义

In [24]:
import pandas as pd
import gurobipy as gp
import numpy as np
model = gp.Model("crop_planting")
acre_yield = pd.read_csv('data/AcreYield.csv',header=None)                         # 每亩产量
expected_sales = pd.read_csv('data/expectedSales.csv',encoding='gbk')  # 预期销售量
planting_cost = pd.read_csv('data/PlantingCost.csv',header=None)                   # 种植成本
unit_price = pd.read_csv('data/unitPrice.csv',encoding='gbk')          # 销售价格
land_area = pd.read_csv('data/landArea.csv',encoding='gbk')            # 地块面积

In [25]:
# 农作物集合、地块集合、年份集合
I = range(0,41)   # 农作物集合
J = range(0,54)  # 地块集合
T = range(0,7)   # 年份集合 (24 - 30)

In [26]:
# 地块子集 (平旱地, 梯田等)
J1 = range(0,6)
J2 = range(6,20)
J3 = range(20,26)
J4 =range(26,34)
J5 = range(34,50)
J6 = range(50,54)
#农作物子集 (粮食, 蔬菜等)
I1 = range(0,15)
i_16 = 15
I2 = range(16,34)
I3 = range(34,37)
I4 =range(37,41)
I5 = [0,1,2,3,4,16,17,18]


In [27]:
# 读取2023年的种植情况
x_0 = pd.read_csv('data/planting1.csv', header=None)  # 第一季
y_0 = pd.read_csv('data/planting2.csv', header=None)  # 第二季
# 决策变量：x_{tij} 和 y_{tij} (第 t 年第 1、2 季度作物 i 在地块 j 的种植面积)
x = model.addVars(T, I, J, name="x", lb=0)
y = model.addVars(T, I, J, name="y", lb=0)

# 辅助变量 z_1i 和 z_2i 用于定义销售量约束
z_1 = model.addVars(T, I, name="z_1", lb=0)
z_2 = model.addVars(T, I, name="z_2", lb=0)

# 约束条件
**1. 平旱地、梯田和山坡地适宜每年种植一季粮食类作物(除了水稻):**

    a. 平旱地、梯田和山坡地每年第一季不能种除粮食(除水稻)之外的其他农作物：
    $$x_{tij} = 0,\quad \forall i \in \complement_II_1,\quad \forall j \in J_1\cup J_2\cup J_3,\quad \forall t \in T$$

In [28]:
for t in T:
    for j in list(J1) + list(J2) + list(J3):  # 平旱地、梯田、山坡地
        for i in I:  # 作物集合
            if i not in I1:  # 非粮食类作物
                model.addConstr(x[t, i, j] == 0, name=f"constraint1_a_{t}_{i}_{j}")

    b. 平旱地、梯田和山坡地每年第二季不种任何农作物：
$$y_{tij} = 0,\quad \forall i \in I,\quad \forall j\in J_1\cup J_2\cup J_3,\quad \forall t\in T$$    

In [29]:
for t in T:
    for j in list(J1) + list(J2) + list(J3):  # 平旱地、梯田、山坡地
        for i in I:  # 作物集合
            model.addConstr(y[t, i, j] == 0, name=f"constraint1_b_{t}_{i}_{j}")

**2. 水浇地适宜每年种植一季水稻或第一季种蔬菜 （A）第二季度种蔬菜(B)：**

    a. 水浇地每年第二季不能种除蔬菜（B）之外的其他农作物：
        $$y_{tij} = 0,\quad \forall i\in \complement_II_3,\quad \forall j \in J_4,\quad \forall t\in T$$

In [30]:
for t in T:
    for j in J4:  # 水浇地
        for i in I:  # 作物集合
            if i not in I3:  # 非蔬菜类(B)作物
                model.addConstr(y[t, i, j] == 0, name=f"constraint2_a_{t}_{i}_{j}")

    b.水浇地每年第二季只能在蔬菜(B)的三种蔬菜(大白菜、白萝卜、红萝卜)中选一个种:
$$\left\{\begin{matrix}
y_{t35j}y_{t36j}=0
 \\
y_{t35j}y_{t37j}=0
 \\
y_{t36j}y_{t37j}=0
\end{matrix}\right.,\quad \forall t\in T,\forall j\in J$$

In [31]:
for t in T:
    for j in J4:  # 水浇地
        model.addConstr(y[t, 34, j] * y[t, 35, j] == 0, name=f"constraint2_b1_{t}_35_36_{j}")
        model.addConstr(y[t, 35, j] * y[t, 36, j] == 0, name=f"constraint2_b2_{t}_35_37_{j}")
        model.addConstr(y[t, 35, j] * y[t, 36, j] == 0, name=f"constraint2_b3_{t}_36_37_{j}")

    c. 水浇地每年第一季不能种植除水稻和蔬菜（A）之外的其他作物：
        $$x_{tij}=0,\quad \forall i \in \complement_I(i_{16}\cup I_2),\quad \forall J \in J_4,\quad \forall t \in T$$

In [32]:
temp1 = [i_16] + list(I2)
for t in T:
    for j in J4:  # 水浇地
        for i in I:  # 作物集合
            if i not in temp1:  # 非水稻和蔬菜(A)类作物
                model.addConstr(x[t, i, j] == 0, name=f"constraint2_c_{t}_{i}_{j}")

    d.水浇地每年在第一季度要么种水稻要么种蔬菜，第一季度如果种了水稻，第二季度就不能种蔬菜：
        $$x_{ti_{16}j}(\sum_{i_a\in I2}x_{ti_aj}+\sum_{i_b\in I3}y_{ti_bj})=0,\quad \forall j \in J_4,\quad \forall t \in T$$

In [33]:
for t in T:
    for j in J4:  # 水浇地
        # 确保水稻和蔬菜不能同时种植
        model.addConstr(x[t, 16, j] * (gp.quicksum(x[t, i, j] for i in I2) + gp.quicksum(y[t, i, j] for i in I3)) == 0,
                        name=f"constraint_rice_or_vegetables_{t}_{j}")

**3. 普通大棚适宜每年第一季种蔬菜(A),第二季食用菌：**

    a.普通大棚每年第一季不能种除蔬菜(A)之外的其他作物：
        $$x_{tij}=0,\quad \forall i \in \complement_II_2,\quad \forall j\in J_5,\quad \forall t\in T$$

In [34]:
for t in T:
    for j in J5:  # 普通大棚
        for i in I:  # 作物集合
            if i not in I2:  # 非蔬菜类(A)作物
                model.addConstr(x[t, i, j] == 0, name=f"constraint3_a_{t}_{i}_{j}")

    b.普通大棚每年第二季不能种除食用菌之外的其他作物：
        $$y_{tij}=0,\quad \forall i \in \complement_II_4,\quad \forall j\in J_5,\quad \forall t\in T$$

In [35]:
for t in T:
    for j in J5:  # 普通大棚
        for i in I:  # 作物集合
            if i not in I4:  # 非食用菌类作物
                model.addConstr(y[t, i, j] == 0, name=f"constraint3_b_{t}_{i}_{j}")

**4. 智慧大棚适宜每年种植两季蔬菜(A):**
    
    a.智慧大棚每年第一季不能种除蔬菜(A)之外的其他作物：
            $$x_{tij}=0,\quad \forall i \in \complement_II_2,\quad \forall j\in J_6,\quad \forall t\in T$$

In [36]:
for t in T:
    for j in J6:  # 智慧大棚
        for i in I:  # 作物集合
            if i not in I2:  # 非蔬菜类(A)作物
                model.addConstr(x[t, i, j] == 0, name=f"constraint4_a_{t}_{i}_{j}")

    b.智慧大棚每年第二季不能种除蔬菜(A)之外的其他作物：
        $$y_{tij}=0,\quad \forall i \in \complement_II_2,\quad \forall j\in J_6,\quad \forall t\in T$$

In [37]:
for t in T:
    for j in J6:  # 智慧大棚
        for i in I:  # 作物集合
            if i not in I2:  # 非蔬菜类(A)作物
                model.addConstr(y[t, i, j] == 0, name=f"constraint4_b_{t}_{i}_{j}")

**5. 每种作物在同一地块（含大棚）不能连续重茬种植：**
    
    a.23年的种植情况对24年的影响:
    $$\left\{\begin{matrix}
    x_{0ij}x_{1ij}=0
 \\
    y_{0ij}y_{1ij}=0
\end{matrix}\right.,\quad \forall i\in I,\quad \forall j\in J$$

$$
\left\{\begin{matrix}x_{0ij}y_{1ij}=0 
\\
y_{0ij}x_{1ij}=0\end{matrix}\right.,\quad \forall i\in I,\quad \forall j\in J6,$$

In [38]:
for j in J:  # 地块
    for i in I:  # 作物
        model.addConstr(x_0[i][j] * x[0, i, j] == 0, name=f"constraint_rotation_2023_1_{i}_{j}")
        model.addConstr(y_0[i][j] * y[0, i, j] == 0, name=f"constraint_rotation_2023_2_{i}_{j}")
for j in J6:
    for i in I:
        model.addConstr(y_0[i][j] * x[0, i, j] == 0, name=f"constraint_rotation_2023_3_{i}_{j}")
        model.addConstr(x_0[i][j] * y[0, i, j] == 0, name=f"constraint_rotation_2023_4_{i}_{j}")

    b.对于除智慧大棚的地块：
    $$\left\{\begin{matrix}x_{tij}x_{(t+1)ij}=0
    \\
    y_{tij}y_{(t+1)ij}=0
    \end{matrix}\right.,\quad \forall i\in I,\quad \forall j\in \complement_JJ_6,\quad \forall t\in \complement_Tt_7$$

    c.对于智慧大棚：
    $$\left\{\begin{matrix}
x_{tij}(y_{tij}+x_{(t+1)ij}+y_{(t+1)ij})=0 
\\
y_{tij}(x_{tij}+x_{(t+1)ij}+x_{(t+1)ij})=0 
\\
x_{(t+1)ij}(x_{tij}+y_{tij}+y_{(t+1)ij})=0 
\\
y_{(t+1)ij}(x_{tij}+y_{tij}+x_{(t+1)ij})=0
\end{matrix}\right.,\quad \forall i\in I,\quad \forall j\in J6,\quad \forall t\in 0\cup \complement_Tt_7$$
    

In [39]:
for t in range(0,6):
    for j in J:
        for i in I:
            if j not in J6:
                model.addConstr(x[t, i, j]*x[t+1, i, j] == 0,name=f"constraint_rotationX_t_{t}_i_{i}_j_{j}")
                model.addConstr(y[t, i, j]*y[t+1, i, j] == 0,name=f"constraint_rotationY_t_{t}_i_{i}_j_{j}")
            else:
                model.addConstr(x[t, i, j]*(y[t, i, j]+x[t+1, i, j]+y[t+1, i, j]) == 0,name=f"constraint_rotationX_t_{t}_i_{i}_j_{j}")
                model.addConstr(y[t, i, j]*(x[t, i, j]+x[t+1, i, j]+y[t+1, i, j]) == 0,name=f"constraint_rotationY_t_{t}_i_{i}_j_{j}")
                model.addConstr(x[t+1, i, j]*(x[t, i, j]+y[t, i, j]+y[t+1, i, j]) == 0,name=f"constraint_rotationX_t_{t}_i_{i}_j_{j}")
                model.addConstr(y[t+1, i, j]*(x[t, i, j]+y[t, i, j]+x[t+1, i, j]) == 0,name=f"constraint_rotationY_t_{t}_i_{i}_j_{j}")

**6. 每个地块（含大棚）的所有土地三年内至少种植一次豆类作物：**

    a.23年的种植情况对24、25年的影响:
    $$\sum_{i\in I_5}\left\{x_{0ij}+y_{0ij}+x_{(1)ij}+y_{(1)ij}+x_{(2)ij}+y_{(2)ij}\right\}\ge s_j,\quad \forall j\in J$$

In [40]:
for j in J:
    model.addConstr(
        gp.quicksum(x_0[i][j] + y_0[i][j] + x[1, i, j] + y[1, i, j] + x[2, i, j] + y[2, i, j] for i in I5) >= land_area.iloc[j,1],
        name=f"three_year_bean_2023_to_2025_j_{j}"
    )

    b.24-30年：
    $$\sum_{i\in I_5}\left\{x_{tij}+y_{tij}+x_{(t+1)ij}+y_{(t+1)ij}+x_{(t+2)ij}+y_{(t+2)ij}\right\}\ge s_j,\quad \forall j\in J,\quad \forall t\in \complement_T(t_6,t_7)$$

In [41]:
for t in range(0, 5):  # 从 t=1 (2024) 到 t=5 (2028)，确保三年周期
    for j in J:
        model.addConstr(
            gp.quicksum(x[t, i, j] + y[t, i, j] + x[t+1, i, j] + y[t+1, i, j] + x[t+2, i, j] + y[t+2, i, j] for i in I5) >= land_area.iloc[j,1],
            name=f"three_year_bean_rotation_t_{t}_j_{j}"
        )

**7. 每块土地的种植面积和的一般约束:**

In [42]:
for t in T:
    for j in J:
        model.addConstr(gp.quicksum(x[t, i, j] for i in I) <= land_area.iloc[j, 1], name=f"land_limit_x_{t}_{j}")
        model.addConstr(gp.quicksum(y[t, i, j] for i in I) <= land_area.iloc[j, 1], name=f"land_limit_y_{t}_{j}")


# 目标函数
**定义$z_{1ti}$与$z_{2ti}$**
$$z_{1ti} = min \left\{\sum_{j\in J}(x_{tij}+y_{tij})o_{ij},d_i\right\},\quad \forall i\in I,\quad \forall t\in T$$
$$z_{2ti} = max\left\{\sum_{j\in J}(x_{tij}+y_{tij})o_{ij}-d_i，0 \right\},\quad \forall i \in I,\quad \forall t\in T$$

In [43]:
for t in T:
    for i in I:
        model.addConstr(z_1[t,i] <= gp.quicksum((x[t,i,j] + y[t,i,j]) * acre_yield.iloc[j, i] for j in J), name=f"constraint_z11_{t}_{i}")
        model.addConstr(z_1[t,i] <= expected_sales.iloc[i, 1], name=f"constraint_z12_{t}_{i}")

In [44]:
for t in T:
    for i in I:
        model.addConstr(z_2[t,i] >= gp.quicksum((x[t,i,j] + y[t,i,j]) * acre_yield.iloc[j, i] for j in J) - expected_sales.iloc[i, 1],name=f"constraint_z21_{t}_{i}")
#z_2在创建的时候设置了lb=0

In [45]:
for t in T:
    for i in I:
        model.addConstr(z_2[t,i]+z_1[t,i] == gp.quicksum((x[t,i,j]+y[t,i,j])*acre_yield.iloc[j,i] for j in J), name=f"constraint_z_{t}_{i}")

    1.超出预期销售额的滞销：
$$maxZ=\sum_{t\in T}\sum_{i\in I}\left\{z_{1ti}p_i-\sum_{j\in J}(x_{tij}+y_{tij}c)c_{ij} \right\}$$

model.setObjective(
    gp.quicksum(
        gp.quicksum(
            z_1[t, i]*unit_price.iloc[i, 1]-gp.quicksum((x[t,i,j]+y[t,i,j])*planting_cost.iloc[j, i] for j in J)
            for i in I
        )
        for t in T
    ),
    sense=gp.GRB.MAXIMIZE
)

    2.超出预期销售额的半价出售
$$maxZ = \sum_{t\in T}\sum_{i\in I}[z_{1i}p_i+z_{2i}0.5p_i-\sum_{j\in J}(x_{tij}+y_{tij}c)c_{ij}]$$

In [46]:
model.setObjective(
    gp.quicksum(
        gp.quicksum(
            z_1[t, i]*unit_price.iloc[i, 1] + 0.5*z_2[t, i]*unit_price.iloc[i, 1] - gp.quicksum((x[t, i, j] + y[t, i, j])*planting_cost.iloc[j, i] for j in J)
            for i in I
        )
        for t in T
    ),
    sense=gp.GRB.MAXIMIZE
)

In [47]:
model.setParam('PoolSolutions', 10)

model.optimize()

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: AMD Ryzen 7 7840HS w/ Radeon 780M Graphics, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 30546 rows, 31570 columns and 112362 nonzeros
Model fingerprint: 0xd349123b
Model has 28760 quadratic constraints
Coefficient statistics:
  Matrix range     [3e-01, 2e+04]
  QMatrix range    [1e+00, 1e+00]
  Objective range  [8e-01, 1e+04]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e-01, 2e+05]
Presolve removed 28806 rows and 23718 columns

Continuous model is non-convex -- solving as a MIP

Presolve added 2961 rows and 0 columns
Presolve removed 0 rows and 2540 columns
Presolve time: 0.05s
Presolved: 33507 rows, 29030 columns, 97322 nonzeros
Variable types: 7852 continuous, 21178 integer (21178 binary)
Found heuristic solution: objective 5.045718e+07

Root relaxation: objective 5.917139e+07, 5298 ite

In [48]:
model.objVal

58661541.47392271

In [49]:
V = model.getVars()[:len(x)]
variable_values = [v.X for v in V]  
reshape_array=np.array(variable_values).reshape(len(T), len(I), len(J))
result = []
for i in range(len(T)):
    result_temp = pd.DataFrame(reshape_array[i].transpose())
    result.append(result_temp)

for i in range(len(T)):
    temp_df = pd.DataFrame(result[i])
    # temp_df.to_csv(f'result/result_c{i}.csv', index=False, header=False)

In [50]:
V = model.getVars()[len(x):len(x)+len(y)]
variable_values = [v.X for v in V]
reshape_array=np.array(variable_values).reshape(len(T), len(I), len(J))
result = []
for i in range(len(T)):
    result_temp = pd.DataFrame(reshape_array[i].transpose())
    result.append(result_temp)
for i in range(len(T)):
    temp_df = pd.DataFrame(result[i])
    # temp_df.to_csv(f'result/result_d{i}.csv', index=False, header=False)

In [51]:
# with open('model/model0.pkl', 'wb') as f:
#     pickle.dump(result, f)
# model.write('model/model0.lp')


In [52]:
def calculate_variance_with_loop(df):
    variance_results = {}
    for column in df.columns:
        # 获取当前列
        non_zero_indices = df[column][df[column] != 0].index
        # 如果不为零的单元格少于两个，方差为0
        if len(non_zero_indices) < 2:
            variance_results[column] = 0
        else:
            # 计算索引的方差
            variance_results[column] = np.var(non_zero_indices)
    # 将结果转化为一个 pandas Series
    return pd.Series(variance_results).sum()

In [53]:
for k in range(model.SolCount):
    model.setParam(gp.GRB.Param.SolutionNumber, k)
    print(f"\nSolution {k + 1}:")
    V = model.getVars()[:len(x)]
    variable_values = [v.X for v in V]  # 提取所有变量的 X 值
    reshape_array=np.array(variable_values).reshape(len(T), len(I), len(J))
    for t in range(len(T)):
        result_temp = pd.DataFrame(reshape_array[t].transpose())
        gap = calculate_variance_with_loop(result_temp)
        print(gap)



Solution 1:
261.9905051587302
560.7824206349205
402.71049042175156
450.0072222222222
429.5440972222222
448.4613548752834
1635.4217716736239

Solution 2:
261.9905051587302
560.7824206349205
402.71049042175156
450.0072222222222
429.5440972222222
448.4613548752834
1635.4217716736239

Solution 3:
261.9905051587302
560.7824206349205
402.71049042175156
450.0072222222222
429.5440972222222
448.4613548752834
1635.4217716736239

Solution 4:
261.9905051587302
560.7824206349205
402.71049042175156
450.0072222222222
429.5440972222222
448.4613548752834
1635.4217716736239

Solution 5:
261.9905051587302
560.7824206349205
402.71049042175156
450.0072222222222
429.5440972222222
448.4613548752834
1635.4217716736239

Solution 6:
261.9905051587302
560.7824206349205
402.71049042175156
450.0072222222222
429.5440972222222
448.4613548752834
1635.4217716736239

Solution 7:
261.9905051587302
560.7824206349205
402.71049042175156
450.0072222222222
429.5440972222222
448.4613548752834
1635.4217716736239

Solution 8:
